## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-15-08-48-59 +0000,bbc,Labour face tough fight in Gorton and Denton b...,https://www.bbc.com/news/articles/cd9gx0751kzo...
1,2026-02-15-08-44-54 +0000,nyt,"Live Updates: In Munich, Rubio Calls Europe a ...",https://www.nytimes.com/live/2026/02/14/world/...
2,2026-02-15-07-56-40 +0000,nypost,Retired FBI agent urges rapid DNA testing in N...,https://nypost.com/2026/02/15/us-news/retired-...
3,2026-02-15-07-46-20 +0000,bbc,Last chance for pensioners to get free air fryers,https://www.bbc.com/news/articles/cgrz0p9n5k0o...
4,2026-02-15-07-24-08 +0000,nypost,"Australian strip club chair thrower loses job,...",https://nypost.com/2026/02/15/world-news/austr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2523/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
79,new,13
415,trump,12
12,europe,10
141,ice,9
10,rubio,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
132,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...,60
37,2026-02-15-02-00-00 +0000,wsj,"Inflation is easing, jobs are holding up, and ...",https://www.wsj.com/economy/federal-reserve-so...,39
74,2026-02-14-22-04-10 +0000,nypost,Marco Rubio slams UN at Munich Security Confer...,https://nypost.com/2026/02/14/world-news/sec-o...,39
114,2026-02-14-17-02-08 +0000,nypost,200K protesters fight Iran’s Islamic Republic ...,https://nypost.com/2026/02/14/world-news/200k-...,39
78,2026-02-14-21-23-29 +0000,nypost,Americans think World War III is likely within...,https://nypost.com/2026/02/14/world-news/ameri...,38


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
132,60,2026-02-14-15-01-39 +0000,nypost,Marco Rubio calls on Europe to join new world ...,https://nypost.com/2026/02/14/world-news/marco...
114,39,2026-02-14-17-02-08 +0000,nypost,200K protesters fight Iran’s Islamic Republic ...,https://nypost.com/2026/02/14/world-news/200k-...
15,35,2026-02-15-04-40-22 +0000,nypost,Woman dies after plunging through ice near Cap...,https://nypost.com/2026/02/14/us-news/cape-cod...
2,32,2026-02-15-07-56-40 +0000,nypost,Retired FBI agent urges rapid DNA testing in N...,https://nypost.com/2026/02/15/us-news/retired-...
34,32,2026-02-15-02-04-02 +0000,bbc,Russia killed opposition leader Alexei Navalny...,https://www.bbc.com/news/articles/cwyk4lz4e3eo...
16,27,2026-02-15-04-23-28 +0000,latimes,Southern California sky is lit up by Valentine...,https://www.latimes.com/california/story/2026-...
32,25,2026-02-15-02-11-00 +0000,wsj,Anthropic’s artificial-intelligence tool Claud...,https://www.wsj.com/politics/national-security...
62,25,2026-02-14-23-06-13 +0000,nypost,TSA agents working without pay as funding for ...,https://nypost.com/2026/02/14/us-news/tsa-agen...
38,23,2026-02-15-01-53-24 +0000,nypost,Florida couple proves it’s never too late to f...,https://nypost.com/2026/02/14/us-news/florida-...
39,23,2026-02-15-01-52-11 +0000,nypost,"Barack Obama says aliens are ‘real, but I have...",https://nypost.com/2026/02/14/us-news/obama-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
